In [350]:
import pandas as pd
import numpy as np
from statistics import geometric_mean

import seaborn as sns
import matplotlib.pyplot as plt

from collections import OrderedDict
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_X_y

from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.dummy import DummyClassifier

from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif


from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

In [351]:
## Features data set
ml = pd.read_csv('features.csv')
ml = ml.drop(['Unnamed: 0'], axis = 1)
ml

,NAME,TIME W/ CUTS,CUTS OFF_FIRST ROUND,PRESOLVING TIME W/ CUTS,PRESOLVING TIME W/O CUTS,TIME FIRST ROUND,NODES W/ CUTS,NODES W/O CUTS,INITIAL LP W/ CUTS,INITIAL LP W/O CUTS,...,linknaps_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars,presol_constr,Time Label
0,10teams.mps.gz,10.997772,13.736838,0.057449,0.059797,1.350119,30.0,276.0,2.000000e+01,2.000000e+01,...,0.0,0.0,0.478261,0.026087,1.63347,0.000000,7.377759,0.000000,5.347108,1
1,22433.mps.gz,2.535930,0.823464,0.046082,0.025466,0.172883,1.0,5.0,2.124053e+04,2.124053e+04,...,0.0,0.0,1.000000,0.040121,0.00000,0.015269,6.061457,0.000000,5.288267,0
2,23588.mps.gz,5.527370,2.540436,0.027812,0.031946,0.140688,576.0,981.0,7.649866e+03,7.649866e+03,...,0.0,0.0,1.000000,0.073409,0.00000,0.026683,5.908083,0.000000,4.919981,0
3,30_70_45_05_100.mps.gz,282.653795,258.647469,1.986556,1.625184,18.567687,46.0,201.0,-5.336000e+03,-5.336000e+03,...,0.0,0.0,0.149627,0.000354,3.72884,0.000000,9.284334,0.000000,9.396654,0
4,30_70_45_095_100.mps.gz,743.997969,75.233190,1.994498,2.143665,19.518728,201.0,1.0,-6.132000e+03,-6.132000e+03,...,0.0,0.0,0.164139,0.000339,3.78902,0.000000,9.303375,0.000000,9.435562,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,uct-subprob.mps.gz,3212.414116,18000.370434,0.121225,0.167001,0.360338,75300.0,2159025.8,-2.000000e+00,-2.000000e+00,...,0.0,0.0,0.874303,0.002280,0.90309,0.000000,7.703459,0.000000,7.541683,1
422,umts.mps.gz,1079.840992,11605.745144,0.179507,0.230250,0.892776,144227.0,3106370.0,1.460426e+06,1.460426e+06,...,0.0,0.0,0.012094,0.001738,6.14613,0.027856,7.411556,0.039879,7.541152,1
423,unitcal_7.mps.gz,940.799926,10628.686922,6.937303,6.065447,64.953906,705.0,196989.0,1.028415e+06,1.028415e+06,...,0.0,0.0,0.356669,0.000101,4.30103,0.795567,10.083139,0.000000,10.642349,1
424,var-smallemery-m6j6.mps.gz,7171.038360,10325.784605,8.873553,8.342061,79.559820,685889.0,836578.0,-1.697219e+03,-1.697219e+03,...,0.0,0.0,0.011181,0.011306,0.00000,0.039372,8.113127,0.000000,9.259321,1


In [352]:
df_train_1 = pd.read_csv('training_first_round_plot.csv')
df_train_1 = df_train_1.filter(['NAME', 'TIME W/ CUTS', 'CUTS OFF_FIRST ROUND'], axis = 1)
df_train_1

,NAME,TIME W/ CUTS,CUTS OFF_FIRST ROUND
0,neos-5140963-mincio.mps.gz,16296.812996,17910.845801
1,neos8.mps.gz,16.780323,72.890510
2,neos-4391920-timok.mps.gz,299.384161,847.066375
3,30_70_45_095_98.mps.gz,76.557754,40.224768
4,wachplan.mps.gz,1434.605408,1779.156125
...,...,...,...
335,ns1644855.mps.gz,860.987882,853.974045
336,neos-3065804-namu.mps.gz,11.837614,11.855551
337,ic97_tension.mps.gz,20.023839,32.066948
338,bppc8-02.mps.gz,0.677820,0.169876


In [353]:
affected = []
for index, row in df_train_1.iterrows():
    
    if row['TIME W/ CUTS'] < 0.8*row['CUTS OFF_FIRST ROUND'] or row['CUTS OFF_FIRST ROUND'] < 0.8*row['TIME W/ CUTS']:
        affected.append(1)
        
    else:
        affected.append(0)
        
        
df_train_1['Affected'] = affected
df_train_1['Affected'].value_counts()[1]
df_train_1 = df_train_1.drop(['TIME W/ CUTS', 'CUTS OFF_FIRST ROUND'], axis = 1)
df_train_1

set_1_train = df_train_1.merge(ml, on = ['NAME'], how = 'left')
set_1_train

,NAME,Affected,TIME W/ CUTS,CUTS OFF_FIRST ROUND,PRESOLVING TIME W/ CUTS,PRESOLVING TIME W/O CUTS,TIME FIRST ROUND,NODES W/ CUTS,NODES W/O CUTS,INITIAL LP W/ CUTS,...,linknaps_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars,presol_constr,Time Label
0,neos-5140963-mincio.mps.gz,0,16296.812996,17910.845801,0.045498,0.017984,0.027525,5624805.0,5527433.2,8.877000e+03,...,0.00000,0.000000,0.847826,0.023126,3.37675,0.971717,5.278115,0.000000,5.214936,1
1,neos8.mps.gz,1,16.780323,72.890510,16.042202,22.825316,16.406966,1.0,1.0,-4.390700e+04,...,0.04896,0.000000,0.497884,0.000291,3.43136,0.248909,10.048280,0.000173,10.740995,1
2,neos-4391920-timok.mps.gz,1,299.384161,847.066375,68.027085,33.449583,556.933012,1.0,1.0,2.282219e-03,...,0.00000,0.000000,0.000623,0.000034,0.00000,1.150550,11.449410,0.000000,12.125811,1
3,30_70_45_095_98.mps.gz,1,76.557754,40.224768,1.683165,2.150924,13.898364,1.0,1.0,-5.967000e+03,...,0.00000,0.000000,0.160532,0.000338,3.77706,0.000000,9.303649,0.000000,9.430359,0
4,wachplan.mps.gz,0,1434.605408,1779.156125,1.102859,1.619734,2.617340,67520.0,98056.0,-2.400000e+01,...,0.00000,0.000000,0.018030,0.017120,0.00000,0.000000,7.741534,0.000434,6.582025,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,ns1644855.mps.gz,0,860.987882,853.974045,26.047749,40.978283,0.000000,1.0,1.0,-2.000000e+05,...,0.00000,0.000000,0.493882,0.001717,0.00000,0.642555,10.315564,0.000000,10.328690,0
336,neos-3065804-namu.mps.gz,0,11.837614,11.855551,7.056363,6.946319,0.000000,1.0,1.0,-3.689729e+08,...,0.00000,0.000000,0.666768,0.036965,7.77789,1.758950,8.464003,0.000000,8.095599,1
337,ic97_tension.mps.gz,1,20.023839,32.066948,0.039291,0.034147,0.067639,22278.0,83938.0,2.606000e+03,...,0.00000,0.000000,0.987461,0.009230,0.00000,1.003350,6.481577,0.006126,5.736572,1
338,bppc8-02.mps.gz,1,0.677820,0.169876,0.028787,0.030815,0.052751,125.0,177.0,3.600000e+01,...,0.00000,0.016949,0.559322,0.320500,0.00000,1.311860,5.438079,0.000000,4.060443,0


In [354]:
reps = [10 if val==1 else 1 for val in df_train_1.Affected]
df_train_2 = df_train_1.loc[np.repeat(df_train_1.index.values, reps)]
df_train_2

set_2_train = df_train_2.merge(ml, on = ['NAME'], how = 'left')
set_2_train

,NAME,Affected,TIME W/ CUTS,CUTS OFF_FIRST ROUND,PRESOLVING TIME W/ CUTS,PRESOLVING TIME W/O CUTS,TIME FIRST ROUND,NODES W/ CUTS,NODES W/O CUTS,INITIAL LP W/ CUTS,...,linknaps_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars,presol_constr,Time Label
0,neos-5140963-mincio.mps.gz,0,16296.812996,17910.845801,0.045498,0.017984,0.027525,5624805.0,5527433.2,8877.0,...,0.00000,0.0,0.847826,0.023126,3.37675,0.971717,5.278115,0.000000,5.214936,1
1,neos8.mps.gz,1,16.780323,72.890510,16.042202,22.825316,16.406966,1.0,1.0,-43907.0,...,0.04896,0.0,0.497884,0.000291,3.43136,0.248909,10.048280,0.000173,10.740995,1
2,neos8.mps.gz,1,16.780323,72.890510,16.042202,22.825316,16.406966,1.0,1.0,-43907.0,...,0.04896,0.0,0.497884,0.000291,3.43136,0.248909,10.048280,0.000173,10.740995,1
3,neos8.mps.gz,1,16.780323,72.890510,16.042202,22.825316,16.406966,1.0,1.0,-43907.0,...,0.04896,0.0,0.497884,0.000291,3.43136,0.248909,10.048280,0.000173,10.740995,1
4,neos8.mps.gz,1,16.780323,72.890510,16.042202,22.825316,16.406966,1.0,1.0,-43907.0,...,0.04896,0.0,0.497884,0.000291,3.43136,0.248909,10.048280,0.000173,10.740995,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2513,neos-1337307.mps.gz,1,245.945124,18002.122776,0.942162,0.864478,2.107710,2479.0,3039680.4,-1482271.0,...,0.01143,0.0,0.000000,0.001907,3.35149,0.012311,7.951559,0.000000,8.645938,1
2514,neos-1337307.mps.gz,1,245.945124,18002.122776,0.942162,0.864478,2.107710,2479.0,3039680.4,-1482271.0,...,0.01143,0.0,0.000000,0.001907,3.35149,0.012311,7.951559,0.000000,8.645938,1
2515,neos-1337307.mps.gz,1,245.945124,18002.122776,0.942162,0.864478,2.107710,2479.0,3039680.4,-1482271.0,...,0.01143,0.0,0.000000,0.001907,3.35149,0.012311,7.951559,0.000000,8.645938,1
2516,neos-1337307.mps.gz,1,245.945124,18002.122776,0.942162,0.864478,2.107710,2479.0,3039680.4,-1482271.0,...,0.01143,0.0,0.000000,0.001907,3.35149,0.012311,7.951559,0.000000,8.645938,1


In [355]:
df_train_3 = df_train_1.loc[df_train_1['Affected'] == 1]
df_train_3

set_3_train = df_train_3.merge(ml, on = ['NAME'], how = 'left')
set_3_train

,NAME,Affected,TIME W/ CUTS,CUTS OFF_FIRST ROUND,PRESOLVING TIME W/ CUTS,PRESOLVING TIME W/O CUTS,TIME FIRST ROUND,NODES W/ CUTS,NODES W/O CUTS,INITIAL LP W/ CUTS,...,linknaps_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars,presol_constr,Time Label
0,neos8.mps.gz,1,16.780323,72.890510,16.042202,22.825316,16.406966,1.0,1.0,-4.390700e+04,...,0.04896,0.000000,0.497884,0.000291,3.43136,0.248909,10.048280,0.000173,10.740995,1
1,neos-4391920-timok.mps.gz,1,299.384161,847.066375,68.027085,33.449583,556.933012,1.0,1.0,2.282219e-03,...,0.00000,0.000000,0.000623,0.000034,0.00000,1.150550,11.449410,0.000000,12.125811,1
2,30_70_45_095_98.mps.gz,1,76.557754,40.224768,1.683165,2.150924,13.898364,1.0,1.0,-5.967000e+03,...,0.00000,0.000000,0.160532,0.000338,3.77706,0.000000,9.303649,0.000000,9.430359,0
3,neos-3004026-krka.mps.gz,1,403.682415,60.166877,0.981425,1.056644,1.844808,13955.0,5663.0,0.000000e+00,...,0.00000,0.000000,0.000000,0.000196,0.00000,0.601173,9.742732,0.007634,9.437077,0
4,neos-3226448-wkra.mps.gz,1,18000.002084,6639.967056,0.271203,0.261297,1307.799684,188.0,44.0,0.000000e+00,...,0.00000,0.000000,0.000000,0.001812,0.00000,0.000000,9.211540,0.000000,7.699842,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,ab51-40-100.mps.gz,1,717.364914,1387.340706,5.161266,5.611407,0.000000,1316.0,1000.0,-3.998180e+12,...,0.00000,0.000000,0.000000,0.000483,6.61842,0.000000,9.447387,0.000000,9.383453,1
238,neos-5106984-jizera.mps.gz,1,12238.856126,19391.656314,61.353719,84.837092,1391.619376,28.0,84.8,8.095000e+02,...,0.00000,0.000000,0.229248,0.000009,0.00000,0.020053,13.057104,0.000000,13.380636,1
239,ic97_tension.mps.gz,1,20.023839,32.066948,0.039291,0.034147,0.067639,22278.0,83938.0,2.606000e+03,...,0.00000,0.000000,0.987461,0.009230,0.00000,1.003350,6.481577,0.006126,5.736572,1
240,bppc8-02.mps.gz,1,0.677820,0.169876,0.028787,0.030815,0.052751,125.0,177.0,3.600000e+01,...,0.00000,0.016949,0.559322,0.320500,0.00000,1.311860,5.438079,0.000000,4.060443,0


In [356]:
df_train_3_unaff = df_train_1.loc[df_train_1['Affected'] == 0]
df_train_3_unaff

set_3_train_unaff = df_train_3_unaff.merge(ml, on = ['NAME'], how = 'left')
set_3_train_unaff

,NAME,Affected,TIME W/ CUTS,CUTS OFF_FIRST ROUND,PRESOLVING TIME W/ CUTS,PRESOLVING TIME W/O CUTS,TIME FIRST ROUND,NODES W/ CUTS,NODES W/O CUTS,INITIAL LP W/ CUTS,...,linknaps_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars,presol_constr,Time Label
0,neos-5140963-mincio.mps.gz,0,16296.812996,17910.845801,0.045498,0.017984,0.027525,5624805.0,5527433.2,8.877000e+03,...,0.000000,0.0,0.847826,0.023126,3.37675,0.971717,5.278115,0.000000,5.214936,1
1,wachplan.mps.gz,0,1434.605408,1779.156125,1.102859,1.619734,2.617340,67520.0,98056.0,-2.400000e+01,...,0.000000,0.0,0.018030,0.017120,0.00000,0.000000,7.741534,0.000434,6.582025,1
2,swath2.mps.gz,0,61.110012,55.268436,0.846663,1.466546,1.638919,1926.0,1333.0,3.344969e+02,...,0.000000,0.0,0.546380,0.005812,0.00000,0.562714,8.764678,0.000000,6.180017,0
3,CMS750_4.mps.gz,0,18000.492354,15887.423821,3.645453,4.177945,4.036673,1.0,155687.0,2.500000e+02,...,0.000000,0.0,0.393505,0.000234,0.00000,0.565802,9.367002,0.000000,9.703816,0
4,map16715-04.mps.gz,0,1890.841437,1634.970929,2.319032,2.764992,11.690022,1657.0,2000.0,-1.675000e+03,...,0.000003,0.0,0.027666,0.000010,0.00000,0.000003,10.197760,0.000000,10.822175,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,neos-799716.mps.gz,0,12.678454,15.602930,2.463649,4.221352,2.933173,1.0,1.0,1.244862e+06,...,0.000000,0.0,0.331065,0.000081,0.00000,2.590340,10.007667,0.000000,10.151870,1
94,qap10.mps.gz,0,26.439974,26.259712,2.135524,2.099088,0.000000,1.0,1.0,3.325662e+02,...,0.000000,0.0,0.000000,0.002410,1.69897,0.000000,8.330864,0.000000,7.506592,0
95,neos-3116779-oban.mps.gz,0,0.025119,0.023967,0.040970,0.030578,0.000000,1.0,1.0,0.000000e+00,...,0.213415,0.0,0.003049,0.012193,0.00000,0.192309,8.545003,0.000000,5.793014,0
96,ns1644855.mps.gz,0,860.987882,853.974045,26.047749,40.978283,0.000000,1.0,1.0,-2.000000e+05,...,0.000000,0.0,0.493882,0.001717,0.00000,0.642555,10.315564,0.000000,10.328690,0


In [357]:
df_test = pd.read_csv('testing_first_round_plot.csv')
df_test = df_test.filter(['NAME', 'TIME W/ CUTS', 'CUTS OFF_FIRST ROUND'], axis = 1)
df_test

,NAME,TIME W/ CUTS,CUTS OFF_FIRST ROUND
0,neos-780889.mps.gz,176.675456,449.053574
1,mzzv11.mps.gz,422.340278,1525.322799
2,chromaticindex32-8.mps.gz,1.025220,3.402795
3,neos22.mps.gz,1.452381,18000.395746
4,ab67-40-100.mps.gz,300.208641,331.249310
...,...,...,...
81,mik-250-20-75-2.mps.gz,12.884775,24.797749
82,30_70_45_095_100.mps.gz,743.997969,75.233190
83,neos-4322846-ryton.mps.gz,818.066564,845.596316
84,ran14x18-disj-8.mps.gz,1658.803667,16537.955252


In [358]:
affected = []
for index, row in df_test.iterrows():
    
    if row['TIME W/ CUTS'] < 0.8*row['CUTS OFF_FIRST ROUND'] or row['CUTS OFF_FIRST ROUND'] < 0.8*row['TIME W/ CUTS']:
        affected.append(1)
        
    else:
        affected.append(0)
        
        
df_test['Affected'] = affected
df_test['Affected'].value_counts()[1]
df_test

,NAME,TIME W/ CUTS,CUTS OFF_FIRST ROUND,Affected
0,neos-780889.mps.gz,176.675456,449.053574,1
1,mzzv11.mps.gz,422.340278,1525.322799,1
2,chromaticindex32-8.mps.gz,1.025220,3.402795,1
3,neos22.mps.gz,1.452381,18000.395746,1
4,ab67-40-100.mps.gz,300.208641,331.249310,0
...,...,...,...,...
81,mik-250-20-75-2.mps.gz,12.884775,24.797749,1
82,30_70_45_095_100.mps.gz,743.997969,75.233190,1
83,neos-4322846-ryton.mps.gz,818.066564,845.596316,0
84,ran14x18-disj-8.mps.gz,1658.803667,16537.955252,1


In [359]:
df_test = df_test.drop(['TIME W/ CUTS', 'CUTS OFF_FIRST ROUND'], axis = 1)
test = df_test.merge(ml, on = ['NAME'], how = 'left')
test

,NAME,Affected,TIME W/ CUTS,CUTS OFF_FIRST ROUND,PRESOLVING TIME W/ CUTS,PRESOLVING TIME W/O CUTS,TIME FIRST ROUND,NODES W/ CUTS,NODES W/O CUTS,INITIAL LP W/ CUTS,...,linknaps_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars,presol_constr,Time Label
0,neos-780889.mps.gz,1,176.675456,449.053574,41.040358,76.790793,82.125212,1.0,1.0,-1.545922e+07,...,0.000000,0.0,0.004735,0.000037,1.34423,0.002259,12.115601,0.000000,11.210603,1
1,mzzv11.mps.gz,1,422.340278,1525.322799,12.828483,19.519242,35.412962,1356.0,8880.0,-2.599800e+04,...,0.000105,0.0,0.000105,0.001384,2.39794,0.054545,9.234057,0.019336,9.158942,1
2,chromaticindex32-8.mps.gz,1,1.025220,3.402795,0.061411,0.062859,0.000000,1.0,249.0,3.000000e+00,...,0.000000,0.0,0.000000,0.001734,0.00000,0.000000,7.742402,0.000000,7.654917,1
3,neos22.mps.gz,1,1.452381,18000.395746,0.144371,0.234904,0.286416,1.0,4782054.0,-1.056000e+06,...,0.000000,0.0,0.253456,0.000730,3.18709,0.933821,8.083329,0.000000,8.557951,1
4,ab67-40-100.mps.gz,0,300.208641,331.249310,5.725706,5.875944,15.087084,73.0,75.0,-4.706400e+12,...,0.000000,0.0,0.000000,0.000490,4.88588,0.000000,9.458138,0.000000,9.400051,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,mik-250-20-75-2.mps.gz,1,12.884775,24.797749,0.038528,0.043530,0.052026,427.0,90650.0,-2.069740e+05,...,0.000000,0.0,0.000000,0.176068,2.54382,1.262980,5.598422,0.574074,4.317488,1
82,30_70_45_095_100.mps.gz,1,743.997969,75.233190,1.994498,2.143665,19.518728,201.0,1.0,-6.132000e+03,...,0.000000,0.0,0.164139,0.000339,3.78902,0.000000,9.303375,0.000000,9.435562,0
83,neos-4322846-ryton.mps.gz,0,818.066564,845.596316,99.861583,92.231158,174.644219,664.0,522.0,4.198484e-01,...,0.000000,0.0,0.001400,0.000040,0.00000,0.002117,11.596347,0.000000,11.561449,1
84,ran14x18-disj-8.mps.gz,1,1658.803667,16537.955252,0.084866,0.091920,0.239374,312024.0,14591597.0,3.444421e+03,...,0.000000,0.0,0.326622,0.045617,2.42160,1.098130,6.222576,0.000000,6.102559,1


SETUP 1

In [360]:
X_train = set_1_train.drop(['NAME', 'Affected', 'TIME W/ CUTS', 'CUTS OFF_FIRST ROUND', 'NODES W/ CUTS', 'NODES W/O CUTS', 
                            'Time Label', 'linsetpart_constr', 'linknaps_constr', 'int_vars', 'linbinpack_constr',
                            'bin_vars', 'presol_constr', 'linsetcov_constr',], axis = 1)
X_test = test.drop(['NAME', 'Affected', 'TIME W/ CUTS', 'CUTS OFF_FIRST ROUND', 'NODES W/ CUTS', 'NODES W/O CUTS', 
                    'Time Label', 'linsetpart_constr', 'linknaps_constr', 'int_vars', 'linbinpack_constr', 'bin_vars', 
                    'presol_constr', 'linsetcov_constr',], axis = 1)

y_train = set_1_train.filter(['Time Label'], axis = 1)
y_test = test.filter(['Time Label'], axis = 1)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
X_train

(340, 19) (340, 1)
(86, 19) (86, 1)


,PRESOLVING TIME W/ CUTS,PRESOLVING TIME W/O CUTS,TIME FIRST ROUND,INITIAL LP W/ CUTS,INITIAL LP W/O CUTS,FIRST ROUND CUT,vars,cont_vars,constr,linsetpack_constr,lincard_constr,lineqknap_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars
0,0.045498,0.017984,0.027525,8.877000e+03,8.877000e+03,8877.000000,5.278115,0.066327,5.214936,0.000000,0.000000,0.0,0.000000,0.847826,0.023126,3.37675,0.971717,5.278115,0.000000
1,16.042202,22.825316,16.406966,-4.390700e+04,-4.390700e+04,-3719.000000,10.053114,0.000000,10.743415,0.000086,0.000259,0.0,0.000000,0.497884,0.000291,3.43136,0.248909,10.048280,0.000173
2,68.027085,33.449583,556.933012,2.282219e-03,2.282219e-03,0.002282,11.449410,0.000639,12.125811,0.000000,0.000000,0.0,0.000000,0.000623,0.000034,0.00000,1.150550,11.449410,0.000000
3,1.683165,2.150924,13.898364,-5.967000e+03,-5.967000e+03,11.500000,9.304741,0.000091,9.431161,0.000000,0.000000,0.0,0.000000,0.160532,0.000338,3.77706,0.000000,9.303649,0.000000
4,1.102859,1.619734,2.617340,-2.400000e+01,-2.400000e+01,-9.000000,8.119994,0.000000,7.347944,0.000000,0.144237,0.0,0.000000,0.018030,0.017120,0.00000,0.000000,7.741534,0.000434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,26.047749,40.978283,0.000000,-2.000000e+05,-2.000000e+05,0.000000,10.315597,0.668874,10.613934,0.000000,0.000000,0.0,0.000000,0.493882,0.001717,0.00000,0.642555,10.315564,0.000000
336,7.056363,6.946319,0.000000,-3.689729e+08,-3.689729e+08,0.000000,8.464003,0.538072,8.095599,0.111280,0.000000,0.0,0.000000,0.666768,0.036965,7.77789,1.758950,8.464003,0.000000
337,0.039291,0.034147,0.067639,2.606000e+03,2.606000e+03,3900.698555,6.555357,0.743954,5.765191,0.000000,0.000000,0.0,0.000000,0.987461,0.009230,0.00000,1.003350,6.481577,0.006126
338,0.028787,0.030815,0.052751,3.600000e+01,3.600000e+01,487.000000,5.446737,0.008621,4.077537,0.000000,0.000000,0.0,0.016949,0.559322,0.320500,0.00000,1.311860,5.438079,0.000000


In [361]:
# Scaling data
scaler = StandardScaler()
# Fit only to the training data
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [362]:
print('NEURAL NETWORKS')
clf = MLPClassifier(solver='lbfgs', batch_size='auto', learning_rate='constant', power_t=0.5, max_iter=500, 
                    shuffle=True, tol=0.00001, verbose=True, 
                    early_stopping=False, validation_fraction=0.1,
                    hidden_layer_sizes=(5, ), activation='relu', alpha=0.001, 
                    learning_rate_init=0.1, random_state=0)
clf.fit(X_train, y_train)
y_train_pred_nn = clf.predict(X_train)
y_test_pred_nn = clf.predict(X_test)

# score_test = clf.score(X_test, y_test)
# score_train = clf.score(X_train, y_train)

score_test = accuracy_score(y_test, y_test_pred_nn)
score_train = accuracy_score(y_train, y_train_pred_nn)

# mse_train = mean_squared_error(y_train, y_train_pred_nn)
# mse_test = mean_squared_error(y_test, y_test_pred_nn)
# print('MSE Test: {}, MSE Train: {}' .format(mse_test, mse_train))
print("Accuracy score test: {}\n".format(score_test))
print("Accuracy score train: {}\n".format(score_train))

# print(classification_report(y_test, y_test_pred_nn, target_names=target_names))
# scores['NN'] = [score, metrics.precision_score(y_test, y_test_pred_nn), metrics.recall_score(y_test, y_test_pred_nn),
#                       metrics.f1_score(y_test, y_test_pred_nn), mse_test, mse_train]


NEURAL NETWORKS


C:\Users\ambar\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Accuracy score test: 0.6976744186046512

Accuracy score train: 0.8176470588235294



In [363]:
set_1_train['Pred'] = y_train_pred_nn
set_1_train_affected = set_1_train.loc[set_1_train['Affected'] == 1]
set_1_train_affected
test['Pred'] = y_test_pred_nn
test_affected = test.loc[test['Affected'] == 1]
# test_affected

train_time_label = set_1_train_affected['Time Label'].values.tolist()
train_pred_label = set_1_train_affected['Pred'].values.tolist()

test_time_label = test_affected['Time Label'].values.tolist()
test_pred_label = test_affected['Pred'].values.tolist()

score_train_affected = accuracy_score(train_time_label, train_pred_label)
score_test_affected = accuracy_score(test_time_label, test_pred_label)

print("Accuracy score test affected: {}\n".format(score_test_affected))
print("Accuracy score train affected: {}\n".format(score_train_affected))

Accuracy score test affected: 0.7164179104477612

Accuracy score train affected: 0.8512396694214877



SETUP 2

In [364]:
X_train = set_2_train.drop(['NAME', 'Affected', 'TIME W/ CUTS', 'CUTS OFF_FIRST ROUND', 'NODES W/ CUTS', 'NODES W/O CUTS', 
                            'Time Label', 'linsetpart_constr', 'linknaps_constr', 'int_vars', 'linbinpack_constr',
                            'bin_vars', 'presol_constr', 'linsetcov_constr',], axis = 1)
X_test = test.drop(['NAME', 'Affected', 'TIME W/ CUTS', 'CUTS OFF_FIRST ROUND', 'NODES W/ CUTS', 'NODES W/O CUTS', 
                    'Time Label', 'linsetpart_constr', 'linknaps_constr', 'int_vars', 'linbinpack_constr', 'bin_vars', 
                    'presol_constr', 'linsetcov_constr', 'Pred'], axis = 1)

y_train = set_2_train.filter(['Time Label'], axis = 1)
y_test = test.filter(['Time Label'], axis = 1)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
X_train

(2518, 19) (2518, 1)
(86, 19) (86, 1)


,PRESOLVING TIME W/ CUTS,PRESOLVING TIME W/O CUTS,TIME FIRST ROUND,INITIAL LP W/ CUTS,INITIAL LP W/O CUTS,FIRST ROUND CUT,vars,cont_vars,constr,linsetpack_constr,lincard_constr,lineqknap_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars
0,0.045498,0.017984,0.027525,8877.0,8877.0,8877.0000,5.278115,0.066327,5.214936,0.000000,0.000000,0.0,0.0,0.847826,0.023126,3.37675,0.971717,5.278115,0.000000
1,16.042202,22.825316,16.406966,-43907.0,-43907.0,-3719.0000,10.053114,0.000000,10.743415,0.000086,0.000259,0.0,0.0,0.497884,0.000291,3.43136,0.248909,10.048280,0.000173
2,16.042202,22.825316,16.406966,-43907.0,-43907.0,-3719.0000,10.053114,0.000000,10.743415,0.000086,0.000259,0.0,0.0,0.497884,0.000291,3.43136,0.248909,10.048280,0.000173
3,16.042202,22.825316,16.406966,-43907.0,-43907.0,-3719.0000,10.053114,0.000000,10.743415,0.000086,0.000259,0.0,0.0,0.497884,0.000291,3.43136,0.248909,10.048280,0.000173
4,16.042202,22.825316,16.406966,-43907.0,-43907.0,-3719.0000,10.053114,0.000000,10.743415,0.000086,0.000259,0.0,0.0,0.497884,0.000291,3.43136,0.248909,10.048280,0.000173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2513,0.942162,0.864478,2.107710,-1482271.0,-1482271.0,-203808.2469,7.951559,0.000000,8.645938,0.000000,0.000000,0.0,0.0,0.000000,0.001907,3.35149,0.012311,7.951559,0.000000
2514,0.942162,0.864478,2.107710,-1482271.0,-1482271.0,-203808.2469,7.951559,0.000000,8.645938,0.000000,0.000000,0.0,0.0,0.000000,0.001907,3.35149,0.012311,7.951559,0.000000
2515,0.942162,0.864478,2.107710,-1482271.0,-1482271.0,-203808.2469,7.951559,0.000000,8.645938,0.000000,0.000000,0.0,0.0,0.000000,0.001907,3.35149,0.012311,7.951559,0.000000
2516,0.942162,0.864478,2.107710,-1482271.0,-1482271.0,-203808.2469,7.951559,0.000000,8.645938,0.000000,0.000000,0.0,0.0,0.000000,0.001907,3.35149,0.012311,7.951559,0.000000


In [365]:
# Scaling data
scaler = StandardScaler()
# Fit only to the training data
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [366]:
print('NEURAL NETWORKS')
clf = MLPClassifier(solver='lbfgs', batch_size='auto', learning_rate='constant', power_t=0.5, max_iter=500, 
                    shuffle=True, tol=0.00001, verbose=True, 
                    early_stopping=False, validation_fraction=0.1,
                    hidden_layer_sizes=(5, ), activation='relu', alpha=0.001, 
                    learning_rate_init=0.1, random_state=0)
clf.fit(X_train, y_train)
y_train_pred_nn = clf.predict(X_train)
y_test_pred_nn = clf.predict(X_test)


set_2_train['Pred'] = y_train_pred_nn
set_2_train = set_2_train.drop_duplicates(subset=['NAME'])


train_time_label = set_2_train['Time Label'].values.tolist()
train_pred_label = set_2_train['Pred'].values.tolist()


score_test = accuracy_score(y_test, y_test_pred_nn)
score_train = accuracy_score(train_time_label, train_pred_label)




print("Accuracy score test: {}\n".format(score_test))
print("Accuracy score train: {}\n".format(score_train))


NEURAL NETWORKS


C:\Users\ambar\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Accuracy score test: 0.6744186046511628

Accuracy score train: 0.7823529411764706



C:\Users\ambar\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [367]:
# Affected instances

set_2_train_affected = set_2_train.loc[set_2_train['Affected'] == 1]
# set_2_train_affected
test['Pred'] = y_test_pred_nn
test_affected = test.loc[test['Affected'] == 1]
# test_affected

train_time_label = set_2_train_affected['Time Label'].values.tolist()
train_pred_label = set_2_train_affected['Pred'].values.tolist()

test_time_label = test_affected['Time Label'].values.tolist()
test_pred_label = test_affected['Pred'].values.tolist()

score_train_affected = accuracy_score(train_time_label, train_pred_label)
score_test_affected = accuracy_score(test_time_label, test_pred_label)

print("Accuracy score test affected: {}\n".format(score_test_affected))
print("Accuracy score train affected: {}\n".format(score_train_affected))

Accuracy score test affected: 0.6716417910447762

Accuracy score train affected: 0.859504132231405



SETUP 3

In [368]:
# Setup 3 to predict real test data
X_train = set_3_train.drop(['NAME', 'Affected', 'TIME W/ CUTS', 'CUTS OFF_FIRST ROUND', 'NODES W/ CUTS', 'NODES W/O CUTS', 
                            'Time Label', 'linsetpart_constr', 'linknaps_constr', 'int_vars', 'linbinpack_constr',
                            'bin_vars', 'presol_constr', 'linsetcov_constr',], axis = 1)
X_test = test.drop(['NAME', 'Affected', 'TIME W/ CUTS', 'CUTS OFF_FIRST ROUND', 'NODES W/ CUTS', 'NODES W/O CUTS', 
                    'Time Label', 'linsetpart_constr', 'linknaps_constr', 'int_vars', 'linbinpack_constr', 'bin_vars', 
                    'presol_constr', 'linsetcov_constr', 'Pred'], axis = 1)

y_train = set_3_train.filter(['Time Label'], axis = 1)
y_test = test.filter(['Time Label'], axis = 1)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
X_train

(242, 19) (242, 1)
(86, 19) (86, 1)


,PRESOLVING TIME W/ CUTS,PRESOLVING TIME W/O CUTS,TIME FIRST ROUND,INITIAL LP W/ CUTS,INITIAL LP W/O CUTS,FIRST ROUND CUT,vars,cont_vars,constr,linsetpack_constr,lincard_constr,lineqknap_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars
0,16.042202,22.825316,16.406966,-4.390700e+04,-4.390700e+04,-3719.000000,10.053114,0.000000,10.743415,0.000086,0.000259,0.0,0.000000,0.497884,0.000291,3.43136,0.248909,10.048280,0.000173
1,68.027085,33.449583,556.933012,2.282219e-03,2.282219e-03,0.002282,11.449410,0.000639,12.125811,0.000000,0.000000,0.0,0.000000,0.000623,0.000034,0.00000,1.150550,11.449410,0.000000
2,1.683165,2.150924,13.898364,-5.967000e+03,-5.967000e+03,11.500000,9.304741,0.000091,9.431161,0.000000,0.000000,0.0,0.000000,0.160532,0.000338,3.77706,0.000000,9.303649,0.000000
3,0.981425,1.056644,1.844808,0.000000e+00,0.000000e+00,0.000000,9.742732,0.000000,9.437077,0.000000,0.000000,0.0,0.000000,0.000000,0.000196,0.00000,0.601173,9.742732,0.007634
4,0.271203,0.261297,1307.799684,0.000000e+00,0.000000e+00,0.000000,9.211540,0.000000,7.699842,0.000000,0.000000,0.0,0.000000,0.000000,0.001812,0.00000,0.000000,9.211540,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,5.161266,5.611407,0.000000,-3.998180e+12,-3.998180e+12,0.000000,9.447387,0.000000,9.383705,0.010426,0.000000,0.0,0.000000,0.000000,0.000483,6.61842,0.000000,9.447387,0.000000
238,61.353719,84.837092,1391.619376,8.095000e+02,8.095000e+02,810.700000,13.060594,0.975726,13.383163,0.000000,0.002219,0.0,0.000000,0.229248,0.000009,0.00000,0.020053,13.057104,0.000000
239,0.039291,0.034147,0.067639,2.606000e+03,2.606000e+03,3900.698555,6.555357,0.743954,5.765191,0.000000,0.000000,0.0,0.000000,0.987461,0.009230,0.00000,1.003350,6.481577,0.006126
240,0.028787,0.030815,0.052751,3.600000e+01,3.600000e+01,487.000000,5.446737,0.008621,4.077537,0.000000,0.000000,0.0,0.016949,0.559322,0.320500,0.00000,1.311860,5.438079,0.000000


In [370]:
# Scaling data
scaler = StandardScaler()
# Fit only to the training data
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [371]:
print('NEURAL NETWORKS')
clf = MLPClassifier(solver='lbfgs', batch_size='auto', learning_rate='constant', power_t=0.5, max_iter=500, 
                    shuffle=True, tol=0.00001, verbose=True, 
                    early_stopping=False, validation_fraction=0.1,
                    hidden_layer_sizes=(5, ), activation='relu', alpha=0.001, 
                    learning_rate_init=0.1, random_state=0)
clf.fit(X_train, y_train)
y_train_pred_nn = clf.predict(X_train)
y_test_pred_nn = clf.predict(X_test)


set_3_train_affected = set_3_train.copy()
set_3_train_affected['Pred'] = y_train_pred_nn

test['Pred'] = y_test_pred_nn
test_affected = test.loc[test['Affected'] == 1]
# test_affected


score_test = accuracy_score(y_test, y_test_pred_nn)
print("Accuracy score test: {}\n".format(score_test))


NEURAL NETWORKS


C:\Users\ambar\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Accuracy score test: 0.5930232558139535



C:\Users\ambar\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [372]:
# Setup 3 to predict unaffected instances
X_train = set_3_train.drop(['NAME', 'Affected', 'TIME W/ CUTS', 'CUTS OFF_FIRST ROUND', 'NODES W/ CUTS', 'NODES W/O CUTS', 
                            'Time Label', 'linsetpart_constr', 'linknaps_constr', 'int_vars', 'linbinpack_constr',
                            'bin_vars', 'presol_constr', 'linsetcov_constr',], axis = 1)
X_test = set_3_train_unaff.drop(['NAME', 'Affected', 'TIME W/ CUTS', 'CUTS OFF_FIRST ROUND', 'NODES W/ CUTS', 'NODES W/O CUTS', 
                    'Time Label', 'linsetpart_constr', 'linknaps_constr', 'int_vars', 'linbinpack_constr', 'bin_vars', 
                    'presol_constr', 'linsetcov_constr'], axis = 1)

y_train = set_3_train.filter(['Time Label'], axis = 1)
y_test = set_3_train_unaff.filter(['Time Label'], axis = 1)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
X_train

(242, 19) (242, 1)
(98, 19) (98, 1)


,PRESOLVING TIME W/ CUTS,PRESOLVING TIME W/O CUTS,TIME FIRST ROUND,INITIAL LP W/ CUTS,INITIAL LP W/O CUTS,FIRST ROUND CUT,vars,cont_vars,constr,linsetpack_constr,lincard_constr,lineqknap_constr,linintknap_constr,linmixbin_constr,nonzeroes,obj_dyn,dynamism_mean,presol_vars,presol_int_vars
0,16.042202,22.825316,16.406966,-4.390700e+04,-4.390700e+04,-3719.000000,10.053114,0.000000,10.743415,0.000086,0.000259,0.0,0.000000,0.497884,0.000291,3.43136,0.248909,10.048280,0.000173
1,68.027085,33.449583,556.933012,2.282219e-03,2.282219e-03,0.002282,11.449410,0.000639,12.125811,0.000000,0.000000,0.0,0.000000,0.000623,0.000034,0.00000,1.150550,11.449410,0.000000
2,1.683165,2.150924,13.898364,-5.967000e+03,-5.967000e+03,11.500000,9.304741,0.000091,9.431161,0.000000,0.000000,0.0,0.000000,0.160532,0.000338,3.77706,0.000000,9.303649,0.000000
3,0.981425,1.056644,1.844808,0.000000e+00,0.000000e+00,0.000000,9.742732,0.000000,9.437077,0.000000,0.000000,0.0,0.000000,0.000000,0.000196,0.00000,0.601173,9.742732,0.007634
4,0.271203,0.261297,1307.799684,0.000000e+00,0.000000e+00,0.000000,9.211540,0.000000,7.699842,0.000000,0.000000,0.0,0.000000,0.000000,0.001812,0.00000,0.000000,9.211540,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,5.161266,5.611407,0.000000,-3.998180e+12,-3.998180e+12,0.000000,9.447387,0.000000,9.383705,0.010426,0.000000,0.0,0.000000,0.000000,0.000483,6.61842,0.000000,9.447387,0.000000
238,61.353719,84.837092,1391.619376,8.095000e+02,8.095000e+02,810.700000,13.060594,0.975726,13.383163,0.000000,0.002219,0.0,0.000000,0.229248,0.000009,0.00000,0.020053,13.057104,0.000000
239,0.039291,0.034147,0.067639,2.606000e+03,2.606000e+03,3900.698555,6.555357,0.743954,5.765191,0.000000,0.000000,0.0,0.000000,0.987461,0.009230,0.00000,1.003350,6.481577,0.006126
240,0.028787,0.030815,0.052751,3.600000e+01,3.600000e+01,487.000000,5.446737,0.008621,4.077537,0.000000,0.000000,0.0,0.016949,0.559322,0.320500,0.00000,1.311860,5.438079,0.000000


In [373]:
# Scaling data
scaler = StandardScaler()
# Fit only to the training data
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [376]:
print('NEURAL NETWORKS')
clf = MLPClassifier(solver='lbfgs', batch_size='auto', learning_rate='constant', power_t=0.5, max_iter=500, 
                    shuffle=True, tol=0.00001, verbose=True, 
                    early_stopping=False, validation_fraction=0.1,
                    hidden_layer_sizes=(5, ), activation='relu', alpha=0.001, 
                    learning_rate_init=0.1, random_state=0)
clf.fit(X_train, y_train)
y_train_pred_nn = clf.predict(X_train)
y_test_pred_nn = clf.predict(X_test)

set_3_train_unaffected = set_3_train_unaff.copy()
set_3_train_unaffected['Pred'] = y_test_pred_nn

set_3_train_original = pd.concat([set_3_train_affected, set_3_train_unaffected])
# set_3_train_original

train_time_label = set_3_train_original['Time Label'].values.tolist()
train_pred_label = set_3_train_original['Pred'].values.tolist()
score_train = accuracy_score(train_time_label, train_pred_label)

print("Accuracy score train: {}\n".format(score_train))

NEURAL NETWORKS


C:\Users\ambar\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Accuracy score train: 0.8323529411764706



C:\Users\ambar\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [377]:
## affected using original affected data sets for train and test as per setup 3

train_time_label = set_3_train_affected['Time Label'].values.tolist()
train_pred_label = set_3_train_affected['Pred'].values.tolist()

test_time_label = test_affected['Time Label'].values.tolist()
test_pred_label = test_affected['Pred'].values.tolist()

score_train_affected = accuracy_score(train_time_label, train_pred_label)
score_test_affected = accuracy_score(test_time_label, test_pred_label)

print("Accuracy score test affected: {}\n".format(score_test_affected))
print("Accuracy score train affected: {}\n".format(score_train_affected))

Accuracy score test affected: 0.5970149253731343

Accuracy score train affected: 0.9338842975206612

